In [ ]:
from huggingace_hub import notebook_login
notebook_login()

In [ ]:
!nvidia-smi -L

In [ ]:
## SETUP THE MODEL

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transfomers import AutoTokenizer,AutoConfig,AutoModelForCasualLM
model=AutoModelForCasualLM.from_pretrained('bigscienc',load_in_8bit=True,device_map='auto')
tokenizer=AutoTokenizer.from_pretrained("bigscience/bloom-7bl")

In [ ]:
## Freezing the original weights

In [ ]:
for param in model.parameters():
    param.requires_grad=False
    if param.ndim==1:
        param.data=param.data.to(torch.float32)
model.gradient_checkpointing_enable()
model.enable_input_require_False()

In [ ]:
class CastOutputToFloat(nn.Sequential):
    def forward(self,x):
        return super().forward(x).to(torch.float32)
model.lm_head=CastOutputToFloat(model.lm_head)


In [ ]:
## Settuping the Lora Parameters

In [ ]:
def print_trainable_parameters(model):
    trainable_params=0
    all_param=0
    for _,param in model.named_parameters():
        all_param+=param.numel()
        if param.requires_grad:
            trainable_params+=param.numel()
        print(f"trainable params:{trainable_params}||all params:{all_param}||trainable%:{100*trainable_params}")


In [ ]:
from peft import LoraConfig,get_peft_model
config=LoraConfig(r=16,lora_alpha=32,lora_droput=0.05,bias="none",task_type="CASUAL_LM")
model=get_peft_model(model,config)
print

In [ ]:
import transformers
from dataset import load_dataset
data=load_dataset("Abirate/english_quote")


In [ ]:
def merge_columns(example):
    example["prediction"]=example["quote"]+"-->"+str(example["tags"])
    return example

data['train']=data['train'].map(merge_columns)
data['train']['prediction'][:5]

In [ ]:
data['train'][0]

In [ ]:
data

In [ ]:
data=data.map(lambda samples:tokenizer(samples['prediction']),batched=True)

In [ ]:
## setuping the training

In [ ]:
trainer=transformers.Trainer(model=model,train_dataset=data['train'],args=transformers.TrainingArguments(per_device_train_batch_size=4,gradient_accumulation_steps=4,warmup_steps=100,max_steps=200,learning_rate=2e-4,fp16=True,logging_steps=1,output_dir='outputs'),data_collector=transformer.DataCollatorForLanguageModeling(tokenizer,mlm=False))
model.config.use_cache=False
trainer.train()

In [ ]:
## save the adapters on to the hub

In [ ]:
model.push_to_hub("sample",use_auth_toke=True,commit_message='basic training',private=True)

In [ ]:
## Load adapters from the hub

In [ ]:
import torch
from peft import PeftModel,PeftConfig
from transformers import AutoModelForCasualLLM,AutoTokenizer
pet_model_id="samwit/bloom-7b-lora-tagger"
config=PeftConfig.from_pretrained(peft_model_id)
tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
## Loading the Lora Model
model=PeftModel.from_pretrained(model,peft_model_id)


In [ ]:
## Inference

In [ ]:
batch=tokenizer("the world is an  oyster -->",return_tensor='pt')
with torch.cuda.amp.autocast():
    output_tokens=model.generate(**batch,max_new_tokens=50)
print('\n \n',tokenizer.decode(output_tokens[0],skip_special_tokens=True))

In [ ]:
https://www.youtube.com/watch?v=Us5ZFp16PaU